# Scheme

Suppose we want to analyze the data at a Music Store 

![chinookschema2](assets/chinookschema2.png)

In [5]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("data_input/chinook.db")

albums = pd.read_sql_query("SELECT * FROM albums", conn)
albums.head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


# How many rows are there in your DataFrame?

In [4]:
artists = pd.read_sql_query("SELECT * FROM artists", conn)
artists.shape[0]

275

In [31]:
pd.read_sql_query("SELECT COUNT(*) FROM artists", conn)

,COUNT(*)
0,275


# We want to limit to 5 rows only and index setting

In [3]:
pd.read_sql_query('''
SELECT * FROM artists LIMIT 5
''', conn, index_col='ArtistId')

,Name
ArtistId,
1,AC/DC
2,Accept
3,Aerosmith
4,Alanis Morissette
5,Alice In Chains


# left Join

In [55]:
albums = pd.read_sql_query('''
SELECT AlbumId, Title, a.Name 
FROM albums 
LEFT JOIN artists as a 
ON a.ArtistId = albums.ArtistId
''', conn)

albums.head()

,AlbumId,Title,Name
0,1,For Those About To Rock We Salute You,AC/DC
1,2,Balls to the Wall,Accept
2,3,Restless and Wild,Accept
3,4,Let There Be Rock,AC/DC
4,5,Big Ones,Aerosmith


# Inspect the last 5 rows of data. Which genre is present in the last 5 rows of our `tracks` DataFrame (Check all that apply)?

In [54]:
tracks = pd.read_sql_query('''
SELECT T.*, 
G.name as GenreName, 
A.Title as AlbumTitle, 
Ar.Name as ArtistName 
FROM tracks as T LEFT JOIN genres as G 
ON  T.GenreId = G.GenreId LEFT JOIN albums as A 
ON  T.AlbumId = A.AlbumId LEFT JOIN artists as Ar 
ON  A.ArtistId = Ar.ArtistId
''', conn, index_col='TrackId')

tracks.tail()


,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice,GenreName,AlbumTitle,ArtistName
TrackId,,,,,,,,,,,
3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99,Classical,Respighi:Pines of Rome,Eugene Ormandy
3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99,Classical,Schubert: The Late String Quartets & String Qu...,Emerson String Quartet
3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99,Classical,Monteverdi: L'Orfeo,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99,Classical,Mozart: Chamber Music,Nash Ensemble
3503,Koyaanisqatsi,347,2,10,Philip Glass,206005,3305164,0.99,Soundtrack,Koyaanisqatsi (Soundtrack from the Motion Pict...,Philip Glass Ensemble


In [35]:
tracks.tail()['GenreName']

TrackId
3499     Classical
3500     Classical
3501     Classical
3502     Classical
3503    Soundtrack
Name: GenreName, dtype: object

# Which are among the top 3 most represented genres in the `tracks` DataFrame?

In [36]:
tracks.GenreName.value_counts().head(3)

Rock     1297
Latin     579
Metal     374
Name: GenreName, dtype: int64

# Most artists price their tracks at 0.99 (`mean`) but there are several artists where the `mean()` is 1.99. Which of the Artist has a mean of 0.99 `UnitPrice`

In [37]:
group_artist = tracks.groupby('ArtistName')[['UnitPrice']].mean()
group_artist

group_artist[group_artist.UnitPrice.round(2)!=0.99]

,UnitPrice
ArtistName,
Aquaman,1.99
Battlestar Galactica,1.99
Battlestar Galactica (Classic),1.99
Heroes,1.99
Lost,1.99
The Office,1.99


# What are the top 5 genres that generated the most profit?

In [57]:
top_genre = pd.read_sql_query('''
SELECT genres.GenreId, genres.Name,SUM(invoices.Total) as Total, SUM(Invoice_Items.quantity) as quantity 
FROM tracks 
LEFT JOIN genres ON genres.GenreId=tracks.GenreId 
LEFT JOIN invoice_items on invoice_items.trackid=tracks.trackid 
LEFT JOIN invoices on invoices.invoiceid=invoice_items.invoiceid 
GROUP BY genres.GenreId 
ORDER BY Total DESC 
LIMIT 5
''', conn)

top_genre

,GenreId,Name,Total,quantity
0,1,Rock,7720.02,835
1,7,Latin,3472.55,386
2,3,Metal,2093.13,264
3,4,Alternative & Punk,1961.66,244
4,19,TV Shows,817.71,47


# we want to query for all invoices where country of the billing address is Germany

In [42]:
germany = pd.read_sql_query('''
SELECT * FROM invoices 
WHERE BillingCountry = 'Germany'
''',conn)

germany.head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,6,37,2009-01-19 00:00:00,Berger Straße 10,Frankfurt,None,Germany,60316,0.99
2,7,38,2009-02-01 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
3,12,2,2009-02-11 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,13.86
4,29,36,2009-05-05 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,1.98


#  we'd like to retrieve all invoices from `Canada` and `USA`

In [43]:
america = pd.read_sql_query('''
SELECT * FROM invoices 
WHERE BillingCountry IN ('USA', 'Canada')
''', conn)

america.head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
1,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
2,13,16,2009-02-19 00:00:00,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,0.99
3,14,17,2009-03-04 00:00:00,1 Microsoft Way,Redmond,WA,USA,98052-8300,1.98
4,15,19,2009-03-04 00:00:00,1 Infinite Loop,Cupertino,CA,USA,95014,1.98


# How many rows are there in `popmusic`?

In [53]:
popmusic = pd.read_sql_query('''
SELECT tracks.*, genres.Name as GenreName 
FROM tracks 
LEFT JOIN genres ON genres.genreid=tracks.genreid 
WHERE genres.Name = 'Pop' AND tracks.UnitPrice=0.99
''',conn,index_col='TrackId')

popmusic.shape[0]

48

# We are reviewing Germany market of the last year sales and would like to retrieve all invoices from the year `2012`.

In [58]:
germany_2012 = pd.read_sql_query('''
SELECT * FROM invoices 
WHERE InvoiceDate >= '2012-01-01' AND 
InvoiceDate <= '2012-12-31'
''',con=conn, parse_dates='InvoiceDate')

germany_2012['InvoiceDate'].describe()

count                      83
unique                     71
top       2012-06-25 00:00:00
freq                        2
first     2012-01-01 00:00:00
last      2012-12-30 00:00:00
Name: InvoiceDate, dtype: object

# Based on the data queried, how many of the customers is working at Apple Inc.?

In [48]:
applecust = pd.read_sql_query('''
SELECT firstname, lastname, email, company,
invoiceid, invoicedate, billingcountry, total 
FROM invoices 
left join customers 
on invoices.CustomerId = customers.CustomerId 
WHERE email like '%@apple%'
''', conn)

applecust.shape[0]

49

# Recall how we retrieved all customers that has the most total invoice in the previous exercise. Say from the information, we'd like to retrieve all the top customers invoice.

In [52]:
customerinv = pd.read_sql_query('''
SELECT invoices.*
FROM invoices 
WHERE invoices.CustomerId IN (
SELECT c.CustomerId FROM Customers as c
LEFT JOIN invoices as i on i.CustomerId = c.CustomerId 
GROUP BY c.CustomerId 
ORDER BY SUM(Total) DESC LIMIT 10)
''', conn)

customerinv.head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,46,6,2009-07-11 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,8.91
1,175,6,2011-02-15 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,1.98
2,198,6,2011-05-20 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,3.96
3,220,6,2011-08-22 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,5.94
4,272,6,2012-04-11 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,0.99


# We want to track the Invoice ID number of 46, 175, 198

In [51]:
customerinv = pd.read_sql_query('''
SELECT * FROM invoices
WHERE InvoiceId IN (46, 175, 198)
''', conn)

customerinv.head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,46,6,2009-07-11 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,8.91
1,175,6,2011-02-15 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,1.98
2,198,6,2011-05-20 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,3.96
